In [10]:
import pandas_datareader.data as pdr
import datetime as dt
import pandas as pd
import numpy as np
days=1500
start_date = '1/1/2010'
end_date = dt.date.today()
tickers  = ['RELIANCE.NS']
ohlcv = pdr.get_data_yahoo(tickers[0],start_date,end_date)
DF = ohlcv.head(days)

In [11]:
df=DF.copy()
#MACD
a=12
b=26
df['MA_Fast'] = df['Adj Close'].ewm(span = a , min_periods = a).mean()
df['MA_Slow'] = df['Adj Close'].ewm(span = b , min_periods = b).mean()
df['MACD'] = df['MA_Fast'] - df['MA_Slow']
#Bollinger
n=20
df['MA'] = df['Adj Close'].rolling(n).mean()
df['BB_up'] = df['Adj Close'].rolling(n).mean() + 2*df['MA'].rolling(n).std()
df['BB_dn'] = df['Adj Close'].rolling(n).mean() - 2*df['MA'].rolling(n).std()
#RSI
n=14
df['delta'] = df['Adj Close'] - df['Adj Close'].shift(1)
df['gain'] = np.where(df['delta']>=0,df['delta'],0)
df['loss'] = np.where(df['delta']<0,abs(df['delta']),0)
avg_gain = []
avg_loss = []
gain = df['gain'].tolist()
loss = df['loss'].tolist()
for i in range(len(df)):
    if i < n:
        avg_gain.append(np.NaN)
        avg_loss.append(np.NaN)
    elif i == n:
        avg_gain.append(df['gain'].rolling(n).mean().tolist()[n])
        avg_loss.append(df['loss'].rolling(n).mean().tolist()[n])
    elif i > n:
        avg_gain.append(((n-1)*avg_gain[i-1] + gain[i])/n)
        avg_loss.append(((n-1)*avg_loss[i-1] + loss[i])/n)
df['avg_gain'] = np.array(avg_gain)
df['avg_loss'] = np.array(avg_loss)
df['RS'] = df['avg_gain']/df['avg_loss']
df['RSI'] = 100 - (100/(1+df['RS']))
#Cleaning dataset
df.drop(['MA_Fast','MA_Slow','MA','delta','gain','loss','avg_gain','avg_loss','RS'],axis=1,inplace=True) #cleaning_indicators_residual
df.drop(['High','Low','Open','Close','Volume'],axis=1,inplace=True) #cleaning_unwanted_data
df.dropna(inplace=True) #Top 38 days has been removed due to NaN 
df.head(5)

,Adj Close,MACD,BB_up,BB_dn,RSI
Date,,,,,
2010-03-02,418.060181,-7.208427,446.349965,403.802107,41.662319
2010-03-03,435.748688,-5.486311,445.609220,403.666937,53.888446
2010-03-04,431.156464,-4.427246,444.717809,403.992826,50.905658
2010-03-05,429.370605,-3.684647,443.268248,404.415526,49.752341
2010-03-08,426.946930,-3.247831,441.775786,405.268058,48.157708


In [13]:
#Lets assume we have a investment amount of Rs 500
portfolio=500
nav = pd.DataFrame(index = DF.tail(days-38).index)
nav = nav.assign(leftover = np.zeros(days-38), stock = np.zeros(days-38))
nav.iloc[0,0] = portfolio

nav.head(5)

,leftover,stock
Date,,
2010-03-02,500.0,0.0
2010-03-03,0.0,0.0
2010-03-04,0.0,0.0
2010-03-05,0.0,0.0
2010-03-08,0.0,0.0


In [14]:
def analyse(df):
    if(df['RSI'][0] < 34 and df['MACD'][0] < 0):
        if(df['Adj Close'][0] < df['BB_dn'][0]):
            return 1
        else:return 0
    elif(df['RSI'][0] > 60 and df['MACD'][0] > 0):
        if(df['Adj Close'][0] > df['BB_up'][0]):
            return -1
        else:return 0
    else:
        return 0
#As seen in historic data of many stocks max. time candle when reaches to 30 it get reversed so I choosed safe 34 
#and same for sell chosen safe 60.

In [15]:
signal = 0
prev_signal = 0
prices = df.copy()
for index, row in nav.iterrows():
  signal = np.sign(signal + analyse(df[index:].head(1)))
  leftover = nav.loc[:index].tail(2).head(1).iloc[0,0]

  if(signal == -1):
    nav.loc[index, 'leftover'] = leftover
    nav.loc[index, 'stock'] = 0
    continue

  if(prev_signal == 0 and signal == 1):
    #BUY
    nav.loc[index, 'leftover'] = leftover -  prices.loc[index][0]
    nav.loc[index, 'stock'] = prices.loc[index][0]

  if(prev_signal == 1 and signal == 1):
    #HOLD
    nav.loc[index, 'leftover'] = leftover
    nav.loc[index, 'stock'] = prices.loc[index][0]
  
  if(prev_signal == 1 and signal == 0):
    #SELL
    nav.loc[index, 'leftover'] = leftover +  prices.loc[index][0]
    nav.loc[index, 'stock'] = 0

  if(prev_signal == 0 and signal == 0):
    #WAIT
    nav.loc[index, 'leftover'] = leftover
    nav.loc[index, 'stock'] = 0
  
  prev_signal = signal

In [16]:
nav.tail(10)

,leftover,stock
Date,,
2016-01-18,763.264526,0.0
2016-01-19,763.264526,0.0
2016-01-20,763.264526,0.0
2016-01-21,763.264526,0.0
2016-01-22,763.264526,0.0
2016-01-25,763.264526,0.0
2016-01-27,763.264526,0.0
2016-01-28,763.264526,0.0
2016-01-29,763.264526,0.0
